In [1]:
import mplfinance as mpf
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
my_file = open('API_KEYS.txt',"r")
API_keys = my_file.read().split()

In [3]:
# Runs through stock data and calls strategies and graphs
def algorithims(ticker,cash):
    # Loads in dataframe and renames columns
    API_key = API_keys[random.randint(0,8)]
    time = TimeSeries(key = API_key,output_format='pandas')
    data = time.get_intraday(symbol=ticker,interval='1min',outputsize='full')[0].rename(columns={"1. open":"Open","2. high":"High","3. low":"Low","4. close":"Close","5. volume":"Volume"})
    
    # Splits dataframe into a list of dataframes of each day
    days = [group[1] for group in data.groupby(data.index.day)]
    num_days = len(days)
    
    # Initialize returns for strategies
    strategy1_returns = np.empty(0)
    strategy2_returns = np.empty(0)
    strategy3_returns = np.empty(0)
    market_returns = np.empty(0)
    
    # Loops through dataframes and performs strategies and graphs
    for df in days:
        # Limits data to only during trading hours
        df = df.loc[(df.index >= f'{df.index.date[0]} 9:30:00')]
        df = df.loc[(df.index >= f'{df.index.date[0]} 4:00:00')]
        
        # Aggregates one minutes 
        df = df.resample('5T').agg({'Open':'first','High':'max','Low':'min','Close':'last','Volume':'sum'})
        
        # Add calculated columns for desired indicators 
        df['VWAP'] = (((df['High']+df['Low']+df['Close'])*df['Volume']/3).cumsum())/df['Volume'].cumsum() 
        df = df.dropna()
        
        # Plot indicators and price
        #vwap = mpf.make_addplot(df['VWAP'])
        #mpf.plot(df,addplot=vwap,type='candle',style='yahoo',volume=True,title=f"{df.index.date[0]} {ticker} VWAP")
        
        # Run strategies and add up returns using numpy arrays
        strategy1_returns = np.append(strategy1_returns,strategy1(df,ticker,cash/num_days))
        strategy2_returns = np.append(strategy2_returns,strategy2(df,ticker,cash/num_days))
        strategy3_returns = np.append(strategy3_returns,strategy3(df,ticker,cash/num_days))
        market_returns = np.append(market_returns,market(df,ticker,cash/num_days))
        
    returns = np.array([strategy1_returns,strategy2_returns,strategy3_returns,market_returns])
    return returns 

In [4]:
# VWAP Cross:
# Long when price crosses above VWAP and short when VWAP goes below
def strategy1(df,ticker,cash):
    # Prints which day this is
    date = df.index.date[0]
    #print(f'{date}: Starting VWAP Cross strategy with ${cash} USD')
    
    # Initialize variables
    i = 0
    cash0 = cash
    shares = 0
    shorts = 0
    transactions = 0
    for index,row in df.iterrows():
        if (not i): 
            # sets initial position based on inital VWAP and price
            long = row['VWAP'] < row['Close'] 
            short = not long
            transactions += 1
            i += 1
            if long:
                shares = cash/row['Close']
                cash = 0
                #print(f"{index.time()}: Buy {shares:.3f} {ticker} shares at {row['Close']} per share")
            if short:
                shorts = cash/row['Close']
                cash = cash0*2
                #print(f"{index.time()}: Open {shorts:.3f} {ticker} short contracts at {row['Close']} per contract")
        
        # Long position 
        if ((row['VWAP'] < row['Close']) and (row['VWAP'] < row['Close']) and short):
            #print(f"{index.time()}: Close {shorts:.3f} {ticker} short contracts at {row['Close']} per contract")
            cash -= shorts*row['Close']
            shares = cash/row['Close']
            shorts = 0
            long = True   
            short = not long
            #print(f"{index.time()}: Buy {shares:.3f} {ticker} shares at {row['Close']} per share")
            transactions += 2
        # Short position
        if ((row['VWAP'] > row['Close']) and (row['VWAP'] > row['Close']) and long):
            #print(f"{index.time()}: Sell {shares:.3f} {ticker} shares at {row['Close']} per share")
            cash = shares*row['Close']*2
            shorts = shares
            shares = 0       
            short = True
            long = not short
            #print(f"{index.time()}: Open {shorts:.3f} {ticker} short contracts at {row['Close']} per contract")
            transactions += 2
        price = row['Close']
    # Exit position and calculate returns
    if short: 
        #print(f"{index.time()}: Close {shorts:.3f} {ticker} short contracts at {price} per contract")
        cash -= shorts*price
    if long:
        #print(f"{index.time()}: Sell {shares:.3f} {ticker} shares at {row['Close']} per share")
        cash = shares*price
    transactions += 1
    #print(f'Cash: {cash}')
    returns = (cash - cash0)/cash0
    #print(f'Return: {returns*100:.2f}%')
    #print(f'Transactions: {transactions}')
    #print('-'*70)
    return cash

In [5]:
# VWAP Mean Reversion: 
# Long at troughs and short at peaks with assumption that price will return to VWAP
def strategy2(df,ticker,cash):
    return cash

In [6]:
# TWAP
def strategy3(df,ticker,cash):
    return cash

In [7]:
def market(df,ticker,cash):
    open_price = df['Open'].loc[f'{df.index.date[0]} 9:30:00']
    shares = cash/open_price
    close_price = df['Close'].loc[f'{df.index.date[0]} 16:00:00']
    return shares*close_price

In [8]:
def print_stock_returns(ticker,returns,cash_per_stock):
    print(ticker)
    for i in range(0,3):
        print(f'Strategy {i+1}: {(np.sum(returns[i]) - cash_per_stock)*100/cash_per_stock:.3f}%')
    print(f'Market: {(np.sum(returns[-1]) - cash_per_stock)*100/cash_per_stock:.3f}%')
    print('-'*50)

In [9]:
def print_summary_returns(all_returns,cash):
    print(all_returns)

In [10]:
# Runs through strategies with list of stocks
def main(stocks,cash):
    returns = np.empty((0,0))
    cash_per_stock = cash/len(stocks)
    for ticker in stocks:
        returns = algorithims(ticker,cash_per_stock)
        #all_returns = np.append(all_returns,returns,axis=0)
        print_stock_returns(ticker,returns,cash_per_stock)
    #print_summary_returns(all_returns,cash)
    return #all_returns

In [11]:
stocks = ['AAPL','GME']
cash = 1100
all_returns = main(stocks,cash)

AAPL
Strategy 1: 0.527%
Strategy 2: 0.000%
Strategy 3: 0.000%
Market: -0.487%
--------------------------------------------------
GME
Strategy 1: 1.484%
Strategy 2: 0.000%
Strategy 3: 0.000%
Market: -0.504%
--------------------------------------------------
